# LLM Information Theory Analysis

This notebook demonstrates how to analyze information-theoretic properties of language model generation, including:
1. **Probability Extraction**: Access token probabilities from the decoder
2. **Intervention**: Modify generation process in real-time
3. **Information Theory**: Compute entropy and Shannon information for each token

## Setup

First, let's import our custom modules and set up the environment.

In [ ]:
import sys
import os

# Add src directory to path
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))

# Import our modules
from probability_extractor import ProbabilityExtractor
from information_theory import (
    compute_entropy,
    compute_shannon_information,
    analyze_token_information,
    compute_varentropy
)
from intervention import InterventionManager

# Import standard libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Imports successful!")

## 1. Load Your Local Llama Model

Update the model name to point to your local Llama model. You can use:
- HuggingFace model names: `"meta-llama/Llama-2-7b-hf"`
- Local paths: `"/path/to/your/model"`
- Quantized versions for lower memory usage (set `load_in_8bit=True` or `load_in_4bit=True`)

In [ ]:
# Configuration
MODEL_NAME = "meta-llama/Llama-2-7b-hf"  # Change this to your model

# For smaller/quantized models:
# extractor = ProbabilityExtractor(MODEL_NAME, load_in_8bit=True)
# extractor = ProbabilityExtractor(MODEL_NAME, load_in_4bit=True)

# Load the model
print(f"Loading model: {MODEL_NAME}")
print("This may take a few minutes...\n")

extractor = ProbabilityExtractor(
    MODEL_NAME,
    device=None,  # Auto-detect CUDA/CPU
    load_in_8bit=False  # Set to True if you have limited VRAM
)

print("\n✓ Model loaded successfully!")

## 2. Extracting Token Probabilities

Let's examine what the model predicts for the next token and see the probability distribution.

In [ ]:
# Define a prompt
prompt = "The capital of France is"

# Get next token probabilities
result = extractor.get_next_token_probabilities(prompt, return_top_k=15)

print(f"Prompt: '{prompt}'\n")
print("Top 15 most likely next tokens:")
print("-" * 50)
for i, (token, prob) in enumerate(result['top_k_tokens'], 1):
    print(f"{i:2d}. '{token}' - Probability: {prob:.4f} ({prob*100:.2f}%)")

# Calculate entropy of the distribution
entropy = compute_entropy(result['probabilities'])
print(f"\nEntropy of next token distribution: {entropy:.4f} bits")
print(f"Varentropy: {compute_varentropy(result['probabilities']):.4f}")

### Visualize the Probability Distribution

In [ ]:
# Plot top-k probabilities
tokens, probs = zip(*result['top_k_tokens'][:10])

plt.figure(figsize=(12, 6))
plt.bar(range(len(tokens)), probs, color='steelblue', alpha=0.7)
plt.xlabel('Token', fontsize=12)
plt.ylabel('Probability', fontsize=12)
plt.title(f'Top 10 Next Token Probabilities for: "{prompt}"', fontsize=14)
plt.xticks(range(len(tokens)), [t.replace(' ', '␣') for t in tokens], rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Generate Text with Full Probability Tracking

Generate a sequence and track the probability of each token.

In [ ]:
# Generate text with probability tracking
generation_prompt = "The theory of relativity states that"

result = extractor.generate_with_probabilities(
    prompt=generation_prompt,
    max_new_tokens=30,
    temperature=0.8,
    return_full_distribution=False  # Set to True if you want full distributions
)

print(f"Prompt: {generation_prompt}")
print(f"\nGenerated text:\n{result['generated_text']}")
print("\n" + "="*70 + "\n")

# Display each token with its probability
print("Token-by-token breakdown:")
print("-" * 70)
for i, (token, prob) in enumerate(zip(result['generated_tokens'], result['token_probabilities']), 1):
    surprisal = compute_shannon_information(prob)
    print(f"{i:2d}. '{token:15s}' | p={prob:.4f} | surprisal={surprisal:.2f} bits")

## 4. Information Theory Analysis

Compute entropy and Shannon information for the generated sequence.

In [ ]:
# Comprehensive information analysis
info_analysis = analyze_token_information(result['token_probabilities'])

print("Information-Theoretic Analysis")
print("=" * 70)
print(f"Mean Surprisal (Average Entropy): {info_analysis['mean_surprisal']:.4f} bits")
print(f"Total Information Content:        {info_analysis['total_information']:.4f} bits")
print(f"Perplexity:                       {info_analysis['perplexity']:.4f}")
print(f"Min Surprisal (most predictable): {info_analysis['min_surprisal']:.4f} bits")
print(f"Max Surprisal (most surprising):  {info_analysis['max_surprisal']:.4f} bits")
print(f"Std Dev of Surprisal:             {info_analysis['std_surprisal']:.4f} bits")

### Visualize Information Content

In [ ]:
# Create a comprehensive visualization
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Token probabilities
ax1 = axes[0]
positions = range(1, len(result['token_probabilities']) + 1)
ax1.plot(positions, result['token_probabilities'], marker='o', linewidth=2, markersize=6, color='steelblue')
ax1.set_xlabel('Token Position', fontsize=12)
ax1.set_ylabel('Probability', fontsize=12)
ax1.set_title('Token Probabilities Across Generation', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 1])

# Plot 2: Surprisal (Shannon Information)
ax2 = axes[1]
surprisals = info_analysis['surprisals']
colors = ['green' if s < info_analysis['mean_surprisal'] else 'orange' if s < info_analysis['mean_surprisal'] + info_analysis['std_surprisal'] else 'red' for s in surprisals]
ax2.bar(positions, surprisals, color=colors, alpha=0.7)
ax2.axhline(y=info_analysis['mean_surprisal'], color='blue', linestyle='--', linewidth=2, label='Mean Surprisal')
ax2.set_xlabel('Token Position', fontsize=12)
ax2.set_ylabel('Surprisal (bits)', fontsize=12)
ax2.set_title('Shannon Information (Surprisal) per Token', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Create token table
df = pd.DataFrame({
    'Position': positions,
    'Token': [t.replace(' ', '␣') for t in result['generated_tokens']],
    'Probability': result['token_probabilities'],
    'Surprisal (bits)': surprisals
})

print("\nDetailed Token Information:")
print(df.to_string(index=False))

## 5. Intervention: Modifying Generation

Now let's intervene in the generation process. We can:
- Force specific tokens at certain positions
- Boost or suppress certain token probabilities
- Apply custom transformations

In [ ]:
# Initialize intervention manager
intervention_manager = InterventionManager(extractor)

print("✓ Intervention Manager initialized")

### Example 1: Force Specific Tokens

In [ ]:
intervention_prompt = "The best programming language is"

# Force specific tokens at positions 0 and 2
forced_tokens = {
    0: " Python",  # Force first token to be "Python"
    2: " its"      # Force third token
}

result_forced = intervention_manager.generate_with_intervention(
    prompt=intervention_prompt,
    max_new_tokens=20,
    forced_tokens=forced_tokens,
    temperature=0.8,
    track_alternatives=True
)

print(f"Prompt: {intervention_prompt}")
print(f"\nGenerated with forced tokens: {result_forced['generated_text']}\n")
print("Intervention Log:")
print("-" * 80)
for entry in result_forced['intervention_log'][:10]:
    if entry['intervention']:
        alt = entry.get('alternative_token', 'N/A')
        alt_prob = entry.get('alternative_probability', 0)
        print(f"Position {entry['position']}: FORCED '{entry['token']}' (p={entry['probability']:.4f})")
        print(f"  → Would have been: '{alt}' (p={alt_prob:.4f})")
    else:
        print(f"Position {entry['position']}: '{entry['token']}' (p={entry['probability']:.4f})")

### Example 2: Boost Certain Tokens

In [ ]:
# Create an intervention that boosts science-related words
science_boost = intervention_manager.create_token_boost_intervention(
    boost_tokens=[' science', ' scientific', ' research', ' study', ' theory'],
    boost_factor=3.0
)

prompt_science = "The most important discovery was"

# Generate with boost
result_boosted = intervention_manager.generate_with_intervention(
    prompt=prompt_science,
    max_new_tokens=25,
    intervention_fn=science_boost,
    temperature=0.9,
    track_alternatives=True
)

print(f"Prompt: {prompt_science}")
print(f"\nGenerated with science boost:\n{result_boosted['generated_text']}")

### Example 3: Suppress Certain Tokens

In [ ]:
# Create an intervention that suppresses common words
suppress_common = intervention_manager.create_token_suppression_intervention(
    suppress_tokens=[' the', ' a', ' an', ' is', ' was', ' are'],
    suppression_strength=0.1
)

prompt_suppress = "Once upon a time there"

result_suppressed = intervention_manager.generate_with_intervention(
    prompt=prompt_suppress,
    max_new_tokens=20,
    intervention_fn=suppress_common,
    temperature=0.8
)

print(f"Prompt: {prompt_suppress}")
print(f"\nGenerated with common word suppression:\n{result_suppressed['generated_text']}")

## 6. Comparative Analysis: Intervention vs. Baseline

Let's compare the information-theoretic properties of intervened vs. baseline generation.

In [ ]:
# Analyze intervention effects
analysis = intervention_manager.analyze_intervention_effects(
    prompt="Artificial intelligence will",
    intervention_fn=science_boost,
    max_new_tokens=15,
    num_samples=3
)

print("Comparative Analysis: Intervention vs Baseline")
print("=" * 80)
print(f"Mean Perplexity (Intervention): {analysis['mean_intervention_perplexity']:.4f}")
print(f"Mean Perplexity (Baseline):     {analysis['mean_baseline_perplexity']:.4f}")
print(f"\nMean Surprisal (Intervention):  {analysis['mean_intervention_entropy']:.4f} bits")
print(f"Mean Surprisal (Baseline):       {analysis['mean_baseline_entropy']:.4f} bits")
print("\n" + "=" * 80)

print("\nSample Generations (Intervention):")
for i, sample in enumerate(analysis['intervention_samples'], 1):
    print(f"{i}. {sample['generated_text'][:100]}...")

print("\nSample Generations (Baseline):")
for i, sample in enumerate(analysis['baseline_samples'], 1):
    print(f"{i}. {sample['generated_text'][:100]}...")

## 7. Custom Intervention Example

Create your own custom intervention function. Here's an example that increases entropy (randomness) for more diverse outputs.

In [ ]:
# Custom intervention: Entropy maximization
def increase_diversity(logits, position, context):
    """
    Increase diversity by flattening the probability distribution.
    """
    # Apply higher temperature to increase entropy
    temperature = 1.5
    return logits / temperature

# Test the custom intervention
prompt_custom = "The future of technology includes"

result_custom = intervention_manager.generate_with_intervention(
    prompt=prompt_custom,
    max_new_tokens=25,
    intervention_fn=increase_diversity,
    temperature=1.0
)

print(f"Prompt: {prompt_custom}")
print(f"\nGenerated with diversity boost:\n{result_custom['generated_text']}")

# Analyze information content
custom_info = analyze_token_information(result_custom['token_probabilities'])
print(f"\nMean surprisal: {custom_info['mean_surprisal']:.4f} bits")
print(f"Perplexity: {custom_info['perplexity']:.4f}")

## 8. Advanced: Conditional Interventions

Apply different interventions based on conditions during generation.

In [ ]:
# Create conditional intervention: boost science words only in first 10 tokens
def first_10_positions(position, context):
    return position < 10

conditional_intervention = intervention_manager.create_conditional_intervention(
    condition_fn=first_10_positions,
    true_intervention=science_boost,
    false_intervention=None  # No intervention after position 10
)

result_conditional = intervention_manager.generate_with_intervention(
    prompt="In the beginning,",
    max_new_tokens=20,
    intervention_fn=conditional_intervention,
    temperature=0.8
)

print("Generated with conditional intervention (boost first 10 tokens only):")
print(result_conditional['generated_text'])

## 9. Experiment: Your Turn!

Now it's your turn to experiment. Try:
- Different prompts
- Different intervention strategies
- Analyzing the information-theoretic properties
- Creating custom intervention functions

In [ ]:
# YOUR EXPERIMENTS HERE
your_prompt = "Enter your prompt here"

# Example: Generate and analyze
# your_result = extractor.generate_with_probabilities(
#     prompt=your_prompt,
#     max_new_tokens=30,
#     temperature=0.8
# )

# your_info = analyze_token_information(your_result['token_probabilities'])
# print(your_info)

## Summary

This notebook demonstrated:

1. ✅ **Probability Extraction**: Accessed decoder probabilities for each token
2. ✅ **Intervention**: Modified generation process with various strategies
3. ✅ **Information Theory**: Computed entropy and Shannon information
4. ✅ **Visualization**: Created insightful plots of token probabilities and surprisal
5. ✅ **Comparative Analysis**: Compared intervened vs baseline generation

### Next Steps

- Experiment with different models
- Try more complex intervention strategies
- Analyze longer sequences
- Export results for further analysis

### Resources

- Source code: `../src/`
- Documentation: `../README.md`
- Examples: `../examples/`